# Find NWM Features from a Polygon Boundary

To run this code you need to upload three different items to the Google Colab file explorer:

1.   Your model boundary as a geojson, geopackage, shapefile, or other geopandas supported file format. If you use a shapefile make sure to upload all of the associated files, not just the one with the extension .shp; this is required for both code blocks.
2.   The file nwm_mainstems.gpkg which contains a selection of the NWM version 4_4_0_0 hydrofabric including all of the feature ids of streams along the main flowline for every HUC8, available in the link below.
3.   The file huc8_us.gpkg which contains all of the huc8 boundaries in the United States, available for download [here](https://byu.box.com/s/le7kyjbf80sfyfijyv1k0ov3go6kjdfe). This is only required for the second code block.

**Important Notes:**

*  Make sure to include the file extension when you type in your input_model_boundary_filename.

*  Please wait for the blue circle on the huc8_us.gpkg download at the bottom to finish before running the HUC8 code. This may take some time.

*  Make sure to download any files from the file explorer that you wish to keep as Google Colab will delete them when the runetime expires.

* Make sure that any uploaded files are in the EPSG:4326 projection to ensure correct interects. Any files not in this projection can be reprojected using the warp (reproject) tool in QGIS or the reproject tool in ArcGIS Pro



In [ ]:
!pip install s3fs zarr -q &> log.log

import os
import geopandas as gpd
import plotly.express as px
import s3fs
import xarray

In [ ]:
# @title Get model Feature IDs from the NWM version 4_4_0_0 hydrofabric using a boundary polygon
# @markdown Wait for the blue upload circle by the nwm_mainstems.gpkg file at the bottom of the files window to disappear before running this code block.
input_model_boundary_filename = 'MSTallahatchieRiver_Q100.geojson' # @param {type:"string"}

# set anon to False if you have a credential file stored on your system
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-east-1'))
store = s3fs.S3Map(root='s3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/chrtout.zarr', s3=s3, check=False)
# Get NWM Metadata including latitude and longitude for reaches
nwm_ds = xarray.open_zarr(store)
df = nwm_ds[['feature_id', 'latitude', 'longitude']].to_dataframe().reset_index()
# Make a geodataframe of the NWM data with the latitude and longitude used to make points
nwm_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude,df.latitude))
nwm_gdf.crs = 'EPSG:4326'
# Read in your model extents
boundary = gpd.read_file(input_model_boundary_filename)
# Buffer your boundary to catch the bottom reach_ids in your extent as the points are for the outlets
#geojson_buffered = boundary.geometry.buffer(0.001)
#geojson_buffered = geojson_buffered.reset_index()
#new_column_order = ['index', 'geometry']
#geojson_buffered = geojson_buffered.set_axis(new_column_order, axis=1)
#geojson_buffered = gpd.GeoDataFrame(geojson_buffered)
# Perform a spatial join and return the reaches in your extents
#reach_ids_gdf = gpd.sjoin(geojson_buffered, nwm_gdf).reset_index()
#reach_ids_gdf = reach_ids_gdf.rename(columns={'index': 'index_reach'})
# Perform a spatial join and return the reaches in your extents
reach_ids_gdf = gpd.sjoin(boundary, nwm_gdf).reset_index()
reach_ids_gdf = reach_ids_gdf.rename(columns={'index': 'index_reach'})
potential_reach_info = reach_ids_gdf[['feature_id','latitude','longitude']]

# Read in the mainstems GeoDataFrame and spatial join it with the model extents, then filter nwm_gdf to get lat and lon
mainstems_gdf = gpd.GeoDataFrame.from_file('nwm_mainstems.gpkg', driver='GPKG')
mainstems_gdf = mainstems_gdf.to_crs('EPSG:4326')
relevant_lvl_paths = gpd.sjoin(boundary, mainstems_gdf.rename(columns={'ID': 'ID_right'}), how='left')
relevant_lvl_paths = relevant_lvl_paths.dissolve(by='ID_right').reset_index()
nwm_features = nwm_gdf[nwm_gdf['feature_id'].isin(relevant_lvl_paths['ID_right'])]
FIM_source_reach_info = nwm_features[['feature_id','latitude','longitude']]
FIM_source_reach_info.to_csv('feature_ids.csv', index=False)
potential_reach_info.to_csv('potential_feature_ids.csv', index=False)
print("""File "feature_ids.csv" was uploaded to the Google Colab file explorer.""")
print("""File "potential_feature_ids.csv" was uploaded to the Google Colab file explorer.
""")

print('\033[1m' + 'Potential Reaches')
# Plot reach coordinates
fig = px.scatter_mapbox(
    potential_reach_info,
    lat="latitude",
    lon="longitude",
    hover_name='feature_id',
    zoom=8,
    height=700
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

print('\033[1m' + 'Selected Mainstem Reaches' + '\033[0m')
print("Use the map below to find the furthest downstream feature_id in your model.")
fig1 = px.scatter_mapbox(
    FIM_source_reach_info,
    lat="latitude",
    lon="longitude",
    hover_name='feature_id',
    zoom=8,
    height=700
)
fig1.update_layout(mapbox_style="open-street-map")
fig1.show()

/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:196: RuntimeWarning: driver GPKG does not support open option DRIVER
  return ogr_read(


File "feature_ids.csv" was uploaded to the Google Colab file explorer.
File "potential_feature_ids.csv" was uploaded to the Google Colab file explorer.

Potential Reaches


Selected Mainstem Reaches
Use the map below to find the furthest downstream feature_id in your model.


In [ ]:
# @title Find Containing HUC8's for Your Model Boundary
# @markdown Wait for the blue upload circle by the HUC8.gpkg file at the bottom of the files window to disappear
# @markdown before running this code block. This may take some time. Make sure to change the boundary filename to match your boundary file
input_model_boundary_filename = 'Okatibbee_boundary1_4326.gpkg' # @param {type:"string"}
# Spatial join the HUC8 file with the input boundary file
huc8_gdf = gpd.read_file('huc8_us.gpkg')
boundary = gpd.read_file(input_model_boundary_filename)
model_huc8_gdf = gpd.sjoin(boundary, huc8_gdf).reset_index()
model_huc8_gdf = model_huc8_gdf.dissolve(by='HUC8').reset_index()
model_huc8_gdf['HUC8'].to_csv('containing_huc8s.csv', index=False)
print("""File "containing_huc8s.csv" was saved to the Google Colab file explorer.""")
model_huc8_gdf[['HUC8','NAME']]

File "containing_huc8s.csv" was saved to the Google Colab file explorer.


,HUC8,NAME
0,03160202,Sucarnoochee
1,03170001,Chunky-Okatibbee
2,03170002,Upper Chickasawhay
3,03170004,Upper Leaf
4,03170005,Lower Leaf
5,03180001,Upper Pearl
